In [13]:
from glob import glob
from pathlib import Path
from os import path
from typing import List
import pandas as pd
import numpy as np
import os

from tqdm.notebook import tqdm
tqdm.pandas()

In [14]:
bks = pd.read_csv('bks.csv')

In [15]:
def instance_list() -> List[str]:
    instances = list()
    instance_folder = os.path.join(os.path.abspath(''), '..', 'data', 'hop-tsiligirides')

    for instance in glob(os.path.join(instance_folder, '*.json')):
        instances.append(os.path.splitext(os.path.basename(instance))[0])

    return instances

In [16]:
algo_names = {
    'frankwolfe': 'Base Model',
    'frankwolfe_liftMTZ': 'Lift',
    'frankwolfe_with_vi1_liftMTZ': 'Lift + (15)',
    'frankwolfe_with_vi1_with_vi2_liftMTZ': 'Lift + (15)--(16)',
    'frankwolfe_with_vi1_with_vi2_with_vi3_liftMTZ': 'Lift + (15)--(17)',
    'frankwolfe_with_vi1_with_vi2_with_vi3_with_vi4_liftMTZ': 'Lift + (15)--(18)'
}

In [17]:
def read_data() -> pd.DataFrame:
    results_path = os.path.join(os.path.abspath(''), '..', 'results', 'run')
    dfs = list()

    for algo_name in algo_names:
        results_glob = f"2022-08-*-{algo_name}-*.txt"
        for results_file in tqdm(glob(os.path.join(results_path, results_glob))):                    
            df = pd.read_csv(results_file)
            df['results_file'] = results_file
            dfs.append(df)

    return pd.concat(dfs)


In [18]:
def get_gap(row: pd.Series) -> float:
    i = row.instance_basename
    bks_row = bks[bks.instance_basename == i]

    if len(bks_row) == 0:
        print(f"No entry found in BKS file for instance {i}")
        raise RuntimeError('Missing BKS value')

    lb = bks_row.iloc[0]['BKS']
    
    try:
        ub = row['obj_bound']
    except KeyError:
        ub = row['obj']

    if ub < lb < ub + 0.1:
        ub = lb # Numerical imprecision
    elif lb >= ub + 0.01:
        print(f"Invalid bounds for {row.instance_basename}: {lb=:.2f}, {ub=:.2f}")

    if np.isnan(ub) or ub == 0:
        return 100
    else:
        return 100 * (ub - lb) / ub

In [19]:
d = read_data()

  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/784 [00:00<?, ?it/s]

  0%|          | 0/784 [00:00<?, ?it/s]

In [20]:
d['pct_gap'] = d.progress_apply(get_gap, axis=1)

  0%|          | 0/4704 [00:00<?, ?it/s]

In [21]:
columns = ['lift_mtz', 'add_vi1', 'add_vi2', 'add_vi3', 'add_vi4']
data = ['pct_gap', 'time_s']

In [22]:
tbl = d.groupby(columns).mean()[data]

In [23]:
tbl.round(3)

pct_gap  time_s
lift_mtz add_vi1 add_vi2 add_vi3 add_vi4                 
False    False   False   False   False     52.748   0.182
True     False   False   False   False     50.810   0.186
         True    False   False   False     22.496   0.785
                 True    False   False     22.496   0.835
                         True    False     22.496   0.883
                                 True      15.914   6.585

In [24]:
n = len(algo_names)

cols = 'rr' * n
print(f"\\begin{{tabular}}{{ll {cols}}}")
print('\\toprule')
print('& & ', end='')

algo_header = list()
for algo_name in algo_names.values():
    algo_header.append(f"\\multicolumn{{2}}{{c}}{{\\textsc{{{algo_name}}}}}")
print(' & '.join(algo_header), end=' \\\\\n')

for start_col in range(3, 2 * (n+1), 2):
    print(f"\\cmidrule(lr){{{start_col}-{start_col+1}}}", end='')
print()

print('\\(\\alpha^{\\text{Tsi}}\\) & \\(\\beta^{\\text{Tsi}}\\) & ', end='')
col_header = ['Gap\% & T (s)' for _ in range(n)]
print(' & '.join(col_header), end=' \\\\\n')

print('\\midrule')

for alpha in sorted(d.tsiligirides_hop_alpha.unique()):
    for beta in sorted(d.tsiligirides_hop_beta.unique()):
        f = d[(d.tsiligirides_hop_alpha == alpha) & (d.tsiligirides_hop_beta == beta)]

        print(f"{alpha} & {beta} & ", end='')

        results = list()
        for algorithm in algo_names.keys():
            df = f[f.algorithm == algorithm]
            results.append(f"{df.pct_gap.mean():.2f}")
            results.append(f"{df.time_s.mean():.1f}")
        print(' & '.join(results), end='\\\\\n')

print('\\midrule')

print('\\multicolumn{2}{l}{Overall} & ', end='')

last_row = list()
for algorithm in algo_names.keys():
    df = d[d.algorithm == algorithm]
    last_row.append(f"{df.pct_gap.mean():.2f}")
    last_row.append(f"{df.time_s.mean():.1f}")
print(' & '.join(last_row), end='\\\\\n')

print('\\bottomrule')
print('\\end{tabular}')


\begin{tabular}{ll rrrrrrrrrrrr}
\toprule
& & \multicolumn{2}{c}{\textsc{Base Model}} & \multicolumn{2}{c}{\textsc{Lift}} & \multicolumn{2}{c}{\textsc{Lift + (15)}} & \multicolumn{2}{c}{\textsc{Lift + (15)--(16)}} & \multicolumn{2}{c}{\textsc{Lift + (15)--(17)}} & \multicolumn{2}{c}{\textsc{Lift + (15)--(18)}} \\
\cmidrule(lr){3-4}\cmidrule(lr){5-6}\cmidrule(lr){7-8}\cmidrule(lr){9-10}\cmidrule(lr){11-12}\cmidrule(lr){13-14}
\(\alpha^{\text{Tsi}}\) & \(\beta^{\text{Tsi}}\) & Gap\% & T (s) & Gap\% & T (s) & Gap\% & T (s) & Gap\% & T (s) & Gap\% & T (s) & Gap\% & T (s) \\
\midrule
0.1 & 2 & 28.42 & 0.1 & 25.13 & 0.1 & 14.99 & 0.7 & 14.99 & 0.7 & 14.99 & 0.8 & 5.59 & 6.0\\
0.1 & 3 & 34.61 & 0.0 & 31.61 & 0.0 & 16.70 & 0.7 & 16.70 & 0.7 & 16.70 & 0.7 & 8.23 & 5.7\\
0.1 & 4 & 40.72 & 0.1 & 38.27 & 0.0 & 20.62 & 0.7 & 20.62 & 0.7 & 20.62 & 0.7 & 12.95 & 6.1\\
0.1 & 5 & 41.38 & 0.1 & 39.06 & 0.1 & 20.86 & 0.7 & 20.86 & 0.7 & 20.86 & 0.7 & 12.96 & 6.4\\
0.2 & 2 & 41.08 & 0.2 & 38.21 & 0.2 & 16